In [2]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [3]:
def find_sheet_name(sheet_names):
    gdp_sheet_names = list(filter(lambda x: "gdp-ss" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    gdp_sheet_names = list(filter(lambda x: "gdp" in x.lower() and "ss" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    gdp_sheet_names = list(filter(lambda x: "gdp" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    if len(sheet_names) == 43:
        return "2"

    raise RuntimeError(sheet_names)

In [4]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "TỔNG SẢN PHẨM TRONG NƯỚC": "TỔNG SỐ",
    "Agriculture, Forestry and Fishing": "Nông, lâm nghiệp và thủy sản",
    "Agriculture": "Nông nghiệp",
    "Forestry": "Lâm nghiệp",
    "Fishery": "Thủy sản",
    "Industry and Construction": "Công nghiệp và xây dựng",
    "Industry": "Công nghiệp",
    "Mining and quarrying": "Khai khoáng",
    "Manufacturing": "Công nghiệp chế biến, chế tạo",
    "Production & supply of electricity, gas, hot water, steam and air-conditioning": "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí",
    "Water supply; sewerage, waste management and remediation activities": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Construction": "Xây dựng",
    "Services": "Dịch vụ",
    "Wholesales, retails, vehicles and motorcycles repairs": "Bán buôn và bán lẻ; sửa chữa ô tô, mô tô, xe máy và xe có động cơ khác",
    "Transportation and storage": "Vận tải, kho bãi",
    "Accommodation & catering services": "Dịch vụ lưu trú và ăn uống",
    "Information & communication": "Thông tin và truyền thông",
    "Finance, banking & insurance": "Hoạt động tài chính, ngân hàng và bảo hiểm",
    "Real estate business ": "Hoạt động kinh doanh bất động sản",
    "Scientific and technical specialty": "Hoạt động chuyên môn, khoa học và công nghệ",
    "Administrative activity and supporting service": "Hoạt động hành chính và dịch vụ hỗ trợ",
    "Activities of Communist Party, socio-political organizations; State management and national defense; compulsory social security": "Hoạt động của Đảng Cộng sản, tổ chức chính trị- xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc",
    "Education and training": "Giáo dục và đào tạo",
    "Health care & social relief": "Y tế và hoạt động trợ giúp xã hội",
    "Art, entertainment and recreation": "Nghệ thuật, vui chơi và giải trí",
    "Other service activities": "Hoạt động dịch vụ khác",
    "Activities of households as employers; undifferentiated goods and services producing activities of households for own use": "Hoạt động làm thuê các công việc trong các hộ gia đình, sản xuất sản phẩm vật chất và dịch vụ tự tiêu dùng của hộ gia đình",
    "Products taxes less subsidies on production": "Thuế sản phẩm trừ trợ cấp sản phẩm",
}

In [17]:
def get_gdp_ss(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df[df.columns[0]] = df[df.columns[0]].fillna(df[df.columns[1]])

    if len(df.columns) == 8:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[4],
                # df.columns[5],
                df.columns[6],
                df.columns[7],
            ]
        )
    elif len(df.columns) == 7:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[3],
                df.columns[4],
                # df.columns[6],
            ]
        )
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[2]])
    elif len(df.columns) == 6 and int(date.split("-")[1]) == 12:
        df = df.drop(
            columns=[df.columns[1], df.columns[2], df.columns[5]]
        )
    else:
        raise RuntimeError(df.columns)

    assert len(df.columns) == 3
    df.columns = ["Ngành", "Tổng số", "YOY"]

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ")
                    | (df[df.columns[0]] == "TOTAL")
                    | (df[df.columns[0]] == "TỔNG SẢN PHẨM TRONG NƯỚC")
                ].index[0]
            )
        )
    )

    df[df["Ngành"] == "TỔNG SẢN PHẨM TRONG NƯỚC"] = "TỔNG SỐ"

    if len(df["Ngành"][df["Ngành"] == "TOTAL"].index) > 0:
        df["Ngành"] = df["Ngành"].map(en_to_vn)

    df = df[pd.to_numeric(df[df.columns[1]], errors="coerce").notnull()]

    if int(date.split("-")[0]) < 2024:
        df["YOY"] = df["YOY"] - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)
    return df


xl = pd.ExcelFile("datas/2016-03-29-Bieu-3-2016.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_gdp_ss(xl, sheet_name, "2016-03-29")

,Ngành,Tổng số,YOY,Date
7,TỔNG SỐ,564254.977792,5.459407,2016-03-29
8,"Nông, lâm nghiệp và thủy sản",66186.440353,-1.231457,2016-03-29
9,Nông nghiệp,48450.710945,-2.69,2016-03-29
10,Lâm nghiệp,3843.082739,6.24,2016-03-29
11,Thủy sản,13892.646669,2.12,2016-03-29
12,Công nghiệp và xây dựng,197758.74802,6.716641,2016-03-29
13,Công nghiệp,169404.655907,6.195507,2016-03-29
14,Khai khoáng,42524.945942,-1.2,2016-03-29
15,"Công nghiệp chế biến, chế tạo",98286.275567,7.9,2016-03-29
16,"Sản xuất và phân phối điện, khí đốt, nước nóng...",24970.347568,13.1,2016-03-29


In [15]:
import os
import contextlib

dfs_gdp_ss_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        if month not in [3, 6, 9, 12]:
            continue

        xl = pd.ExcelFile(file_path)

        df_gdp = get_gdp_ss(
            xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}"
        )

        dfs_gdp_ss_origin.append(df_gdp)
    except Exception as e:
        print(file_name)

In [18]:
dfs_gdp_ss = pd.concat(dfs_gdp_ss_origin, ignore_index=True, axis=0)
dfs_gdp_ss["Date"] = pd.to_datetime(dfs_gdp_ss["Date"], dayfirst=False)
dfs_gdp_ss

,Ngành,Tổng số,YOY,Date
0,TỔNG SỐ,1396659.503877,5.657544,2024-03-29
1,"Nông, lâm nghiệp và thủy sản",145600.125191,2.979307,2024-03-29
2,Nông nghiệp,112214.936244,2.805032,2024-03-29
3,Lâm nghiệp,5908.701034,4.083476,2024-03-29
4,Thủy sản,27476.487913,3.459557,2024-03-29
...,...,...,...,...
1140,Giáo dục và đào tạo,14306.642201,7.82,2014-03-28
1141,Y tế và hoạt động trợ giúp xã hội,6434.305429,7.80182,2014-03-28
1142,"Nghệ thuật, vui chơi và giải trí",3831.353046,9.12,2014-03-28
1143,Hoạt động dịch vụ khác,9223.516492,7.44,2014-03-28


In [19]:
dfs_gdp_ss.to_csv("csv/gdp_ss.csv")

In [ ]:
dfs_gdp_ss['Ngành'].unique()

In [ ]:
# nganh = "TỔNG SỐ"
# dfs_gdp_ss_per_nganh = dfs_gdp_ss[dfs_gdp_ss["Ngành"] == nganh]

# dfs_gdp_ss_per_nganh["PCT_YOY"] = dfs_gdp_ss_per_nganh["Tổng số"].pct_change(-4)
# print(dfs_gdp_ss_per_nganh)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_gdp_ss[
    (
        (dfs_gdp_ss["Ngành"] == "TỔNG SỐ")
        | (dfs_gdp_ss["Ngành"] == "Nông, lâm nghiệp và thủy sản")
        | (dfs_gdp_ss["Ngành"] == "Công nghiệp và xây dựng")
        | (dfs_gdp_ss["Ngành"] == "Dịch vụ")
    )
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_gdp_ss[
    (
        (dfs_gdp_ss["Ngành"] == "Công nghiệp chế biến, chế tạo")
        | (dfs_gdp_ss["Ngành"] == "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí")
    )
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()